In [4]:
// CELL 1 - command to set the blob storage account connection
val scalasecret = "FsnZrS+XU+CObmL7yT2SvdLo4UCzR7JvgdW+HLPJVKLIOu+bnHBdBMaa09sHI6eUXeEPVhk3eGFCYd25chG+3g==";
spark.conf.set("fs.azure.account.key.wagssynap.blob.core.windows.net",scalasecret)

scalasecret: String = FsnZrS+XU+CObmL7yT2SvdLo4UCzR7JvgdW+HLPJVKLIOu+bnHBdBMaa09sHI6eUXeEPVhk3eGFCYd25chG+3g==

In [5]:
// CELL 2 - read a JSON file as Spark DataFrame
val blobcontainer = "wasbs://telemetry@wagssynap.blob.core.windows.net"
val blobpath = "/sample/telemetry-20170515-000000.json"
val jsonFile = spark.read.json( blobcontainer + blobpath)

blobcontainer: String = wasbs://telemetry@wagssynap.blob.core.windows.net
blobpath: String = /sample/telemetry-20170515-000000.json
jsonFile: org.apache.spark.sql.DataFrame = [ambient_pressure: double, ambient_temperature: double ... 6 more fields]

In [6]:
val df = spark.read.option("inferSchema","true").json("wasbs://telemetry@wagssynap.blob.core.windows.net/landing/*.json")

df: org.apache.spark.sql.DataFrame = [ambient_pressure: double, ambient_temperature: double ... 6 more fields]

In [7]:
display(jsonFile)

In [8]:
val df = spark.read.option("inferSchema","true").json("wasbs://telemetry@wagssynap.blob.core.windows.net/landing/*.json")

df: org.apache.spark.sql.DataFrame = [ambient_pressure: double, ambient_temperature: double ... 6 more fields]

In [9]:
display(df)

In [10]:
df.createOrReplaceTempView("telemetry")

In [11]:
%%sql
select count(*) from telemetry

In [12]:
%%sql
select min(ts), max(ts) from telemetry

In [12]:
df.write.mode("overwrite").parquet("wasbs://poutput@wagssynap.blob.core.windows.net/poutput")

Error: org.apache.hadoop.fs.azure.AzureException: com.microsoft.azure.storage.StorageException: This operation is not permitted on a non-empty directory.

In [13]:
// CELL 4 - the parquet file created above as Spark DataFrame
val dfParquet = spark.read.parquet("wasbs://poutput@wagssynap.blob.core.windows.net/poutput")

dfParquet: org.apache.spark.sql.DataFrame = [ambient_pressure: double, ambient_temperature: double ... 6 more fields]

In [14]:
dfParquet.cache()

res12: dfParquet.type = [ambient_pressure: double, ambient_temperature: double ... 6 more fields]

In [15]:
dfParquet.createOrReplaceTempView("telemetry")

In [16]:
%%sql
select count(*) from telemetry

In [17]:
%%sql
select  day(cast(ts  as timestamp)) as tsDay, 
        hour(cast(ts  as timestamp)) as tsHour,
        avg(pressure) as avgPressure, avg(speed) as avgSpeed
from telemetry
group by  tsDay, tsHour
order by  tsDay, tsHour